In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
import altair as alt
from altair_saver import save
import os
import re
from nltk.util import ngrams
from itertools import chain

import numpy as np
import warnings
warnings.simplefilter("ignore")

C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('Data_mesh_publications_cleaned.csv')
df_sample = df. copy()

In [3]:
# def generate_custom_token_pattern(ngram_min, ngram_max):
#     # Ensure no word repetition in n-grams
#     word_pattern = r'\b(\w+)\b(?=.*\b\1\b)'
    
#     # Combine word patterns for specified n-gram range
#     ngram_pattern = r'(?:' + word_pattern + r'\s){' + str(ngram_min - 1) + r',' + str(ngram_max - 1) + r'}' + word_pattern
    
#     return ngram_pattern

# token_pattern = generate_custom_token_pattern(1, 4)

In [4]:
# def extract_topics(text, tfidf_vectorizer, lda_model):
    
#     # TF-IDF Vectorization for the specific document
#     tfidf_data = tfidf_vectorizer.transform([text])
#     tfidf_values = tfidf_data.toarray()
#     feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
#     df_tfidf = pd.DataFrame(data=tfidf_values, columns=feature_names)

#     # Apply LDA
#     doc_topic_prob = lda_model.transform(df_tfidf)

#     # Get the most probable topic for the document
#     most_probable_topic = np.argmax(doc_topic_prob)

#     # Get the top words for the most probable topic with TF-IDF values
#     topic = lda_model.components_[most_probable_topic]
#     top_keywords_idx = topic.argsort()[:-10 - 1:-1]
#     top_keywords = [feature_names[i] for i in top_keywords_idx]
#     top_tfidf_values = [tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]] for word in top_keywords]
# #     top_tfidf_values = [tfidf_data[0, tfidf_vectorizer.vocabulary_[word]] for word in top_keywords]


#     topic_words = list(zip(top_keywords, top_tfidf_values))

#     return most_probable_topic + 1, topic_words

# tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 4), token_pattern=r'\b\w+\b')
# tfidf_data = tfidf_vectorizer.fit_transform(df['Content'])

# # Apply LDA
# lda = LatentDirichletAllocation(n_components=38, random_state=42)
# lda.fit(tfidf_data)


# # Assuming 'df' is your DataFrame with 'pdf_content' column
# df['topic'], df['topic_words'] = zip(*df.apply(lambda row: find_topic_words(row['Content']), axis=1))


In [5]:
import pandas as pd
from gensim import corpora

def find_topic_words(df, num_topics=36, num_words=10):
    """
    Find topic words for each content in the DataFrame using LDA.
    
    Parameters:
    - df: DataFrame with a 'Content' column containing preprocessed (tokenized) texts.
    - num_topics: Number of topics to extract.
    - num_words: Number of words to show for each topic.
    
    Returns:
    - topics_df: DataFrame with topics and their words for each document.
    """
    tokenize_content = df['Content'].apply(lambda x: word_tokenize(x.lower()))

    dictionary = corpora.Dictionary(tokenize_content)

    corpus = [dictionary.doc2bow(text) for text in tokenize_content]
    

    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
    

    doc_topics = [lda_model.get_document_topics(bow) for bow in corpus]
    

    topics_df = pd.DataFrame()
    for n in range(len(df)):
        topic_num, prop_topic = sorted(doc_topics[n], key=lambda x: x[1], reverse=True)[0]  
        wp = lda_model.show_topic(topic_num, topn=num_words)
        topic_keywords = ", ".join([word for word, prop in wp])
        word_probs = [(word, round(prop, 4)) for word, prop in wp]  # Get topic words
        topics_df = topics_df.append(pd.Series([int(topic_num), topic_keywords, word_probs]), ignore_index=True)
    
    topics_df.columns = ['Dominant_Topic', 'Topic_Keywords','Word_Probabilities']
    return topics_df

In [6]:

topics_df = find_topic_words(df)
print(topics_df)

    Dominant_Topic                                     Topic_Keywords  \
0                1  data, localization, architecture, regulation, ...   
1               26  data, product, port, output, example, concept,...   
2               14  data, product, quality, lake, team, architectu...   
3               34  data, fabric, ai, product, architecture, cloud...   
4                1  data, localization, architecture, regulation, ...   
5               34  data, fabric, ai, product, architecture, cloud...   
6               34  data, fabric, ai, product, architecture, cloud...   
7               24  data, architecture, cloud, application, fabric...   
8               17  data, metadata, ai, knowledge, fabric, asset, ...   
9                9  data, ml, ai, learning, dl, feature, problem, ...   
10               2  data, ai, ml, quality, fabric, asset, new, dis...   
11              17  data, metadata, ai, knowledge, fabric, asset, ...   
12               9  data, ml, ai, learning, dl, fea

In [7]:
merged_df = pd.concat([df_sample, topics_df], axis=1)


# Results with filename

In [8]:
result_data = []

# Iterate through each row in the original DataFrame
for _, row in merged_df.iterrows():
    Filename = row['Filename']
    topic_words = row['Word_Probabilities']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data.append({'Filename': Filename, 'topic_word': word, 'idf_value': idf_value})

# Create the result DataFrame from the list
result_df = pd.DataFrame(result_data, columns=['Filename', 'topic_word', 'idf_value'])

# Print the result DataFrame
result_df

,Filename,topic_word,idf_value
0,Breaking Down Data Silos Data Mesh to Achieve ...,data,0.1587
1,Breaking Down Data Silos Data Mesh to Achieve ...,localization,0.0313
2,Breaking Down Data Silos Data Mesh to Achieve ...,architecture,0.0144
3,Breaking Down Data Silos Data Mesh to Achieve ...,regulation,0.0115
4,Breaking Down Data Silos Data Mesh to Achieve ...,region,0.0106
...,...,...,...
375,Utilization of Data Mesh Framework as a Part o...,different,0.0074
376,Utilization of Data Mesh Framework as a Part o...,architecture,0.0073
377,Utilization of Data Mesh Framework as a Part o...,new,0.0062
378,Utilization of Data Mesh Framework as a Part o...,system,0.0057


In [42]:
# Select the top terms for each PDF
top_terms= result_df.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms['idf_value'] = top_terms['idf_value'] + np.random.rand(top_terms.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda= (heatmap + text).properties(width=1000)

# Select the top terms for each PDF



In [43]:
# save(chart_lda, "lda_idf_heatmap.html")
chart_lda

alt.LayerChart(...)

# Result with publication type

In [11]:
result_data_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in merged_df.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['Word_Probabilities']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publication_type.append({'Publication_type': Publication_type, 'topic_word': word, 'idf_value': idf_value})

# Create the result DataFrame from the list
result_df_publication_type = pd.DataFrame(result_data_publication_type, columns=['Publication_type', 'topic_word', 'idf_value'])

# Print the result DataFrame
result_df_publication_type

,Publication_type,topic_word,idf_value
0,conference,data,0.1587
1,conference,localization,0.0313
2,conference,architecture,0.0144
3,conference,regulation,0.0115
4,conference,region,0.0106
...,...,...,...
375,master thesis,different,0.0074
376,master thesis,architecture,0.0073
377,master thesis,new,0.0062
378,master thesis,system,0.0057


In [12]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_publication_type.groupby('Publication_type').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['idf_value'] = top_terms_df_publication_type['idf_value'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publication_type:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 2.0, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publication_type = (heatmap + text).properties(width=2000)
chart_lda_publication_type

alt.LayerChart(...)

# Result with publisher

In [13]:
result_data_publisher = []

# Iterate through each row in the original DataFrame
for _, row in merged_df.iterrows():
    Publisher = row['Publisher']
    topic_words = row['Word_Probabilities']

    # Iterate through each tuple in the list of topic_words
    for word, idf_value in topic_words:
        result_data_publisher.append({'Publisher': Publisher, 'topic_word': word, 'idf_value': idf_value})

# Create the result DataFrame from the list
result_df_publisher = pd.DataFrame(result_data_publisher)


result_df_publisher

,Publisher,topic_word,idf_value
0,IEEE,data,0.1587
1,IEEE,localization,0.0313
2,IEEE,architecture,0.0144
3,IEEE,regulation,0.0115
4,IEEE,region,0.0106
...,...,...,...
375,miscellaneous,different,0.0074
376,miscellaneous,architecture,0.0073
377,miscellaneous,new,0.0062
378,miscellaneous,system,0.0057


In [14]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['idf_value'] = top_terms_df_publisher['idf_value'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.idf_value >= 3.0, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_lda_publisher = (heatmap + text).properties(width=1500)
chart_lda_publisher

alt.LayerChart(...)

# USING ONLY TFIDF

In [15]:
# save(chart_lda_publisher, 'lda_idf_heatmap_pub.html')

In [16]:
def find_top_words_for_each_document(df, num_top_words=10):

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 4), token_pattern=r'\b\w+\b')

    # Fit and transform the PDF content
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])

    # Get feature names (words) from the TF-IDF vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Create a DataFrame with TF-IDF values
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Now, for each document, find the top words based on TF-IDF scores
    top_words_for_each_document = []
    for i, row in enumerate(tfidf_df.iterrows()):
        _, document_tfidf_scores = row
        top_words_index = document_tfidf_scores.argsort()[-num_top_words:][::-1]
        top_words = [(feature_names[index], document_tfidf_scores[index]) for index in top_words_index]
        top_words_for_each_document.append(top_words)

    # Create a new DataFrame with 'pdf_content' and 'top_words'
    result_df = pd.DataFrame({'Filename': df['Filename'],'Publisher': df['Publisher'],'Publication_type': df['Publication_type'], 'Content': df['Content'], 'top_words': top_words_for_each_document})

    return result_df

In [17]:
# Call the function
result_df_tfidf = find_top_words_for_each_document(df)

# Display the result DataFrame
# result_df_tfidf['Publisher'] = df['Publisher']
# result_df_tfidf['Publication_type'] = df['Publication_type']
result_df_tfidf

,Filename,Publisher,Publication_type,Content,top_words
0,Breaking Down Data Silos Data Mesh to Achieve ...,IEEE,conference,abstract data localization law becoming make h...,"[(data, 0.48576178749624277), (localization, 0..."
1,Decentralized Data Governance as Part of a Dat...,IEEE,conference,abstract data socio technical decentralized an...,"[(data, 0.6075525028801096), (data product, 0...."
2,Enterprise Data Strategy A Decentralized Data ...,IEEE,conference,abstract enterprise experience exponential gro...,"[(data, 0.47309061294490373), (zone, 0.0799215..."
3,Finding Your Way Through the Jungle of Big Dat...,IEEE,conference,abstract paper present systematic ofcommon ana...,"[(data, 0.4030458903487308), (archimate, 0.148..."
4,01. Evolution of Data Architecture,Springer,book chapter,look back data architecture developed response...,"[(data, 0.5378273130863123), (edw, 0.163525348..."
5,02. Terminology Data Fabric and Data Mesh,Springer,book chapter,term data fabric data often viewed different c...,"[(data, 0.6233304038524975), (data fabric, 0.2..."
6,03. Data Fabric and Data Mesh Use Case Scenarios,Springer,book chapter,organization realized importance implementing ...,"[(data, 0.6053379645931797), (ai, 0.1252673988..."
7,04. Data Fabric and Data Mesh Business Benefits,Springer,book chapter,defining data fabric underlying data architect...,"[(data, 0.19844434979682793), (driver, 0.15607..."
8,05. Key Data Fabric and Data Mesh Capabilities,Springer,book chapter,introduces key capability concept knowledge ca...,"[(data, 0.2705088127274745), (metadata, 0.2411..."
9,06. Relevant ML and DL Concepts,Springer,book chapter,ai ml dlbroadly speaking ai simulate human per...,"[(data, 0.19295752127265792), (dl, 0.192009270..."


# Results with filename

In [18]:
result_data_tfidf_filename = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Filename = row['Filename']
    top_words = row['top_words']

    # Iterate through each tuple in the list of top_words
    for word, tfidf in top_words:
        result_data_tfidf_filename.append({'Filename': Filename, 'topic_word': word, 'tfidf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_filename = pd.DataFrame(result_data_tfidf_filename)

result_df_tfidf_filename

,Filename,topic_word,tfidf_value
0,Breaking Down Data Silos Data Mesh to Achieve ...,data,0.485762
1,Breaking Down Data Silos Data Mesh to Achieve ...,localization,0.465137
2,Breaking Down Data Silos Data Mesh to Achieve ...,data localization,0.434956
3,Breaking Down Data Silos Data Mesh to Achieve ...,region,0.105720
4,Breaking Down Data Silos Data Mesh to Achieve ...,india,0.092454
...,...,...,...
375,Utilization of Data Mesh Framework as a Part o...,product,0.060086
376,Utilization of Data Mesh Framework as a Part o...,interview,0.059538
377,Utilization of Data Mesh Framework as a Part o...,evans,0.059350
378,Utilization of Data Mesh Framework as a Part o...,different,0.059186


In [19]:
import pandas as pd

# Assuming result_df_3 is your DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)# Show all rows

# Display the entire DataFrame
print(result_df_tfidf_filename)


                                                                                         Filename  \
0        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
1        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
2        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
3        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
4        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
5        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
6        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
7        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization   
8        Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Locali

In [44]:
# Select the top terms for each PDF
top_terms_df_tfidf = result_df_tfidf_filename.groupby('Filename').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_tfidf['tfidf_value'] = top_terms_df_tfidf['tfidf_value'] + np.random.rand(top_terms_df_tfidf.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_tfidf).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('tfidf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tfidf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tfidf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.tfidf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_tfidf = (heatmap + text).properties(width=2000)


In [45]:
# save(chart_tfidf, 'tfidf_heatmap.html')
chart_tfidf

alt.LayerChart(...)

# Result with publisher

In [22]:
result_data_tfidf_publisher = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publisher = row['Publisher']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publisher.append({'Publisher': Publisher, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publisher = pd.DataFrame(result_data_tfidf_publisher)


result_df_tfidf_publisher

,Publisher,top_word,tf_idf_value
0,IEEE,data,0.485762
1,IEEE,localization,0.465137
2,IEEE,data localization,0.434956
3,IEEE,region,0.105720
4,IEEE,india,0.092454
5,IEEE,regulation,0.068774
6,IEEE,indian,0.056811
7,IEEE,border,0.055685
8,IEEE,data aggregation,0.054220
9,IEEE,aggregation,0.049685


In [23]:
result_df_tfidf_publisher.describe()

,tf_idf_value
count,380.000000
mean,0.145445
std,0.122759
min,0.044469
25%,0.074595
50%,0.104570
75%,0.151122
max,0.769915


In [24]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_tfidf_publisher.groupby('Publisher').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['tf_idf_value'] = top_terms_df_publisher['tf_idf_value'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publisher = (heatmap + text).properties(width=1500)
chart_publisher

alt.LayerChart(...)

In [25]:
# save(chart_tfidf_publisher_2, 'tfidf_heatmap_publisher.html')

# Results with Publisher

In [26]:
result_data_tfidf_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publication_type.append({'Publication_type': Publication_type, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publication_type = pd.DataFrame(result_data_tfidf_publication_type)


result_df_tfidf_publication_type

,Publication_type,top_word,tf_idf_value
0,conference,data,0.485762
1,conference,localization,0.465137
2,conference,data localization,0.434956
3,conference,region,0.105720
4,conference,india,0.092454
5,conference,regulation,0.068774
6,conference,indian,0.056811
7,conference,border,0.055685
8,conference,data aggregation,0.054220
9,conference,aggregation,0.049685


In [27]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_tfidf_publication_type.groupby('Publication_type').head(10)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['tf_idf_value'] = top_terms_df_publication_type['tf_idf_value'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publication_type:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publication_type = (heatmap + text).properties(width=1500)
chart_publication_type

alt.LayerChart(...)

In [28]:
from gensim import corpora, models
from gensim.utils import simple_preprocess


df['Content'] = df['Content'].astype(str)


document_topics = []


for index, row in df.iterrows():

    text = simple_preprocess(row['Content'], deacc=True)
    
    # Create a dictionary and corpus for the document
    dictionary = corpora.Dictionary([text])
    corpus = [dictionary.doc2bow(text)]
    
 
    lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, passes=15, random_state=100)
    

    topics = lda_model.print_topics(num_words=3)
    topics_str = ''.join([f"[Topic {topic[0]}: {', '.join(topic[1].split('+')).replace('*','').strip()}]" for topic in topics])
    

    document_topics.append((row['Filename'], topics_str))


In [29]:
topics_df = pd.DataFrame(document_topics, columns=['Filename', 'Topics'])
topics_df

,Filename,Topics
0,Breaking Down Data Silos Data Mesh to Achieve Effective Aggregation in Data Localization,"[Topic 0: 0.003""data"" , 0.002""localization"" , 0.002""region""][Topic 1: 0.154""data"" , 0.043""localization"" , 0.015""regulation""][Topic 2: 0.164""data"" , 0.032""localization"" , 0.014""region""]"
1,Decentralized Data Governance as Part of a Data Mesh Platform Concepts and Approaches,"[Topic 0: 0.063""data"" , 0.030""product"" , 0.011""port""][Topic 1: 0.126""data"" , 0.047""product"" , 0.015""port""][Topic 2: 0.129""data"" , 0.030""product"" , 0.015""port""]"
2,Enterprise Data Strategy A Decentralized Data Mesh Approach,"[Topic 0: 0.039""data"" , 0.007""product"" , 0.005""lake""][Topic 1: 0.130""data"" , 0.018""product"" , 0.013""quality""][Topic 2: 0.004""data"" , 0.002""product"" , 0.002""lake""]"
3,Finding Your Way Through the Jungle of Big Data Architectures,"[Topic 0: 0.007""data"" , 0.004""architecture"" , 0.003""fabric""][Topic 1: 0.004""data"" , 0.002""architecture"" , 0.002""fabric""][Topic 2: 0.123""data"" , 0.036""architecture"" , 0.021""warehouse""]"
4,01. Evolution of Data Architecture,"[Topic 0: 0.143""data"" , 0.035""architecture"" , 0.015""application""][Topic 1: 0.004""data"" , 0.002""architecture"" , 0.002""edw""][Topic 2: 0.004""data"" , 0.002""architecture"" , 0.002""new""]"
5,02. Terminology Data Fabric and Data Mesh,"[Topic 0: 0.004""data"" , 0.002""fabric"" , 0.002""ai""][Topic 1: 0.155""data"" , 0.040""fabric"" , 0.036""ai""][Topic 2: 0.002""data"" , 0.001""fabric"" , 0.001""ai""]"
6,03. Data Fabric and Data Mesh Use Case Scenarios,"[Topic 0: 0.006""data"" , 0.002""fabric"" , 0.002""ai""][Topic 1: 0.131""data"" , 0.023""ai"" , 0.021""fabric""][Topic 2: 0.131""data"" , 0.020""ai"" , 0.017""cloud""]"
7,04. Data Fabric and Data Mesh Business Benefits,"[Topic 0: 0.009""data"" , 0.009""architecture"" , 0.009""driver""][Topic 1: 0.095""data"" , 0.038""architecture"" , 0.030""driver""][Topic 2: 0.009""data"" , 0.009""architecture"" , 0.009""new""]"
8,05. Key Data Fabric and Data Mesh Capabilities,"[Topic 0: 0.006""data"" , 0.004""metadata"" , 0.004""ai""][Topic 1: 0.011""data"" , 0.007""ai"" , 0.006""metadata""][Topic 2: 0.065""data"" , 0.046""metadata"" , 0.034""ai""]"
9,06. Relevant ML and DL Concepts,"[Topic 0: 0.001""data"" , 0.001""ml"" , 0.001""ai""][Topic 1: 0.039""data"" , 0.020""ml"" , 0.018""ai""][Topic 2: 0.021""data"" , 0.012""dl"" , 0.012""learning""]"
